#20IT076 Software Group Project III 
Reddit Web scrapper

Installing PRAW as it is not default installed in colab

In [ ]:
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 7.8 MB/s eta 0:00:00


In [ ]:
import praw
import pandas as pd
import json
from datetime import datetime
import sys

reddit = praw.Reddit(client_id='JnZeKfKqP62GP3CzptSLRg', client_secret='ykOlgXsGgI05joj4cMtyx4Ms7ZhIvg', user_agent='20IT076_SGP')



In [ ]:
reddit

In [ ]:
# Since we are testing this in Colab, we need to simulate entering in command line arguments
# We can test this by using the example command given in the docstring, but we will only ask for 5 messages
sys.argv = input('Please enter the command to run your program: ').split(' ')

# Our arguments will be read in as a list, just like it would be from the command line
print(sys.argv, type(sys.argv))

Please enter the command to run your program: python3 python_api.py umd umdcs terps 100
['python3', 'python_api.py', 'umd', 'umdcs', 'terps', '100'] <class 'list'>


In [ ]:
# When running from the command line, the 'python' argument does not get included, so we need to remove it
sys.argv = sys.argv[1:len(sys.argv)]
print(sys.argv)

['python_api.py', 'umd', 'umdcs', 'terps', '100']


In [ ]:
# Count the system arguments (subreddit names) and subtract 1 since the script name will always be included
arguments = len(sys.argv) - 1
print(arguments)

4


In [ ]:
# Count the system arguments (subreddit names) and subtract 1 since the script name will always be included
arguments = len(sys.argv) - 1
print(arguments)

4


In [ ]:
# If the user forgets to enter in any system arguments, we should exit the program and inform them of the error
if arguments == 0:
    sys.exit("Please provide at least one valid subreddit name. Ex: python reddit_api.py funny memes 100")
    

In [ ]:

# Assign the last system argument to be the number of posts retrieved from each subreddit
last_argument = len(sys.argv) - 1

# Exit the script if no number of posts system argument was provided or it was not provided as an integer
try:
    num_messages = int(sys.argv[last_argument])
    print(num_messages)
    
except:
    sys.exit("Enter the last argument as the number of posts to retrieve. Ex: python reddit_api.py funny memes 100")

100


In [ ]:
# Now we are ready to begin scraping data from the Reddit API
# We can test our scraper by manually providing an argument before we convert the code into a loop
# We will use the first subreddit argument which is at position 1 (remember, lists are 0-based in Python)
position = 1

# Let us make sure we can pass this argument to our Reddit client and get some results
try:
    # Choose to examine a given subreddit
    subreddit_name = sys.argv[position]
    subreddit = reddit.subreddit(subreddit_name)

    # Returns a list-like object with the top (newest = subreddit.new) n submissions to the subreddit (limit of 1000)
    top_subreddit = subreddit.top(limit=num_messages)

# If we pass in an invalid subreddit name like asjhasdasdiugqwdb, we want to handle that error appropriately
except:
    sys.exit("Please confirm that all subreddit names are valid. Ex: python reddit_api.py funny memes 100")

In [ ]:
# Let's take a look at how the data is passed to us from Reddit
print(top_subreddit)

In [ ]:
import pprint
for submission in subreddit.top(limit=1):
    pprint.pprint(vars(submission))

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



{'_comments_by_id': {},
 '_fetched': False,
 '_reddit': <praw.reddit.Reddit object at 0x7f0c29382df0>,
 'all_awardings': [{'award_sub_type': 'GLOBAL',
                    'award_type': 'global',
                    'awardings_required_to_grant_benefits': None,
                    'coin_price': 100,
                    'coin_reward': 0,
                    'count': 3,
                    'days_of_drip_extension': None,
                    'days_of_premium': None,
                    'description': "Shows the Silver Award... and that's it.",
                    'end_date': None,
                    'giver_coin_reward': None,
                    'icon_format': None,
                    'icon_height': 512,
                    'icon_url': 'https://www.redditstatic.com/gold/awards/icon/silver_512.png',
                    'icon_width': 512,
                    'id': 'gid_1',
                    'is_enabled': True,
                    'is_new': False,
                    'name': 'Silver',
   

In [ ]:
# We can see there are attributes like author, title, created, etc., that we can pull
# To store these attributes, we can create a dictionary that we will populate with the values we want
# Each key in the dictionary represents an attribute we will scrape, and the values will be lists of the attributes
reddit_dict = {"author": [],
                "title": [],
                "score": [],
                "id": [], 
                "url": [],
                "comms_num": [],
                "created": [],
                "body": [],
                "subscribers": [],
                "subreddit": []}
print(reddit_dict)

{'author': [], 'title': [], 'score': [], 'id': [], 'url': [], 'comms_num': [], 'created': [], 'body': [], 'subscribers': [], 'subreddit': []}


In [ ]:
# When you loop through a ListingGenerator, it expires. We need to recreate it so that we can finish scraping.
# Choose to examine a given subreddit
subreddit_name = sys.argv[position]
subreddit = reddit.subreddit(subreddit_name)

# Returns a list-like object with the top (newest = subreddit.new) n submissions to the subreddit (limit of 1000)
top_subreddit = subreddit.top(limit=num_messages)
print(top_subreddit)

In [ ]:
# We can iterate through the top subreddit posts and append to the dictionary
for submission in top_subreddit:
    # The author must be converted to a string because of the way it is contained in the Redditor class
    reddit_dict["author"].append(str(submission.author))
    reddit_dict["title"].append(submission.title)
    reddit_dict["score"].append(submission.score)
    reddit_dict["id"].append(submission.id)
    reddit_dict["url"].append(submission.url)
    reddit_dict["comms_num"].append(submission.num_comments)
    reddit_dict["created"].append(submission.created)
    reddit_dict["body"].append(submission.selftext)
    reddit_dict["subscribers"].append(submission.subreddit_subscribers)
    reddit_dict["subreddit"].append(subreddit_name)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
# Let's take a look at how our dictionary is currently formatted
print(reddit_dict)

{'author': ['None', 'kinky-unicorn', 'None', 'riverofdenali', 'kensieeo', 'rankindsouza', 'A_Diseye12', 'UMD_leg_Throwaway', 'CodeRedKing', 'mandarinsta', 'hunterm19', 'tealqueen', 'kapperstick', 'stegotops7', 'i_am_eating_food', 'cubenerd01', 'shade10152001', 'Stankylosaur', 'bmcnealey', 'ifthn', 'centrenahte', 'KyloRensTears', 'lukedestroyer12', 'ifthn', 'ifthn', 'FlyAway012', 'ee-squared', 'expandexplore34', 'PretentiousFucker', 'PoshLagoon', 'Juinyk11', '-JDB-', 'KMART44', 'ItsLiterallyPK', 'kingaiden99', 'mandarinsta', 'heresjohnnybird', 'debater-alt', 'LohBlow', 'None', 'skytlezL', 'Mr_Alfie_Doolittle', 'UMD_leg_Throwaway', 'Froski101', 'thomas_irl0', 'mordebyandrigbecai', 'National_Cheesecake5', 'victorioustin', 'Spadestep', 'engulfing_eukaryote', 'Raisinbran_don', 'Captain_Crunch_Hater', 'fuckingthrowitaway21', 'engulfing_eukaryote', 'BIGLanny_', 'vulcan1999', 'jivson', 'gbayonas', 'YoungTerp2022', 'None', 'Jarboner69', 'Snowmuhgeddon', 'whid_bey', 'TerpRhapsody', 'bstark97', '

In [ ]:
# We can see that the data is there, but it's not very intelligible
# To make our data easier to investigate, we can pass the dictionary into a dataframe for easier viewing
reddit_data = pd.DataFrame(reddit_dict)
reddit_data.head()

,author,title,score,id,url,comms_num,created,body,subscribers,subreddit
0,None,"For Karate KNES144Q, we have to record ourselv...",1511,jl5osc,https://v.redd.it/394ovkk3vaw51,56,1.604093e+09,,41729,umd
1,kinky-unicorn,UMD Under a Thunderstorm ⛈⚡️,1078,hwt715,https://i.redd.it/20olbi48rpc51.jpg,26,1.595557e+09,,41729,umd
2,None,My INST633 prof told us to create OC posts for...,1027,ku7wm4,https://i.redd.it/y34tjot3tfa61.jpg,25,1.610255e+09,,41729,umd
3,riverofdenali,Basically my entire first semester here,995,j4nn4e,https://i.redd.it/nzxqalaicyq51.jpg,15,1.601762e+09,,41729,umd
4,kensieeo,Warning for everyone,925,l3dayh,https://www.reddit.com/r/UMD/comments/l3dayh/w...,28,1.611413e+09,I know textbooks are expensive but never go to...,41729,umd


In [ ]:
# The formatting of the created column is not very intelligible. We can change it to ISO from UNIX/UTC
def get_date(created):
    return datetime.utcfromtimestamp(created).isoformat() + '+00:00'

# Apply the above function to the "created" column, and then rename it to timestamp for easier understanding
reddit_data["created"] = reddit_data["created"].apply(get_date)
reddit_data.rename(columns = {"created" : "timestamp"}, inplace = True)

# Examine the dataframe and confirm that our data is structured the way we want
reddit_data

,author,title,score,id,url,comms_num,timestamp,body,subscribers,subreddit
0,None,"For Karate KNES144Q, we have to record ourselv...",1511,jl5osc,https://v.redd.it/394ovkk3vaw51,56,2020-10-30T21:30:34+00:00,,41729,umd
1,kinky-unicorn,UMD Under a Thunderstorm ⛈⚡️,1078,hwt715,https://i.redd.it/20olbi48rpc51.jpg,26,2020-07-24T02:08:56+00:00,,41729,umd
2,None,My INST633 prof told us to create OC posts for...,1027,ku7wm4,https://i.redd.it/y34tjot3tfa61.jpg,25,2021-01-10T05:02:05+00:00,,41729,umd
3,riverofdenali,Basically my entire first semester here,995,j4nn4e,https://i.redd.it/nzxqalaicyq51.jpg,15,2020-10-03T22:00:13+00:00,,41729,umd
4,kensieeo,Warning for everyone,925,l3dayh,https://www.reddit.com/r/UMD/comments/l3dayh/w...,28,2021-01-23T14:46:33+00:00,I know textbooks are expensive but never go to...,41729,umd
...,...,...,...,...,...,...,...,...,...,...
95,CreedBratton__,Is this a good freshman schedule? I got a 5 on...,528,w6dplv,https://i.redd.it/31uibe6yqdd91.png,32,2022-07-23T20:45:48+00:00,,41729,umd
96,Joshmamama,You know how we always have leftover utensils ...,532,k8awk6,https://i.redd.it/ybab8fanfp361.jpg,17,2020-12-07T05:56:12+00:00,,41729,umd
97,RobotReptar,My grandfather was UMD Class of 1962. Here is ...,531,ttc5n0,https://i.ibb.co/nLb3J8v/UMD-1958.jpg,27,2022-03-31T23:00:30+00:00,,41729,umd
98,None,"As the semester ramps up, relieve some stress ...",526,is5rc2,https://www.reddit.com/r/UMD/comments/is5rc2/a...,28,2020-09-13T20:13:41+00:00,>!pop!< >!pop!< >!pop!< >!pop!< >!pop!< >!pop!...,41729,umd


In [ ]:
# Convert the dataframe back to a dictionary for rapid exporting to JSON (dataframes are slow)
reddit_dict = reddit_data.to_dict('records')

# This results in a series of key-value pairs that looks very similar to JSON
reddit_dict

[{'author': 'None',
  'title': 'For Karate KNES144Q, we have to record ourselves practicing kicks. I accidentally kicked my chair.',
  'score': 1511,
  'id': 'jl5osc',
  'url': 'https://v.redd.it/394ovkk3vaw51',
  'comms_num': 56,
  'timestamp': '2020-10-30T21:30:34+00:00',
  'body': '',
  'subscribers': 41729,
  'subreddit': 'umd'},
 {'author': 'kinky-unicorn',
  'title': 'UMD Under a Thunderstorm ⛈⚡️',
  'score': 1078,
  'id': 'hwt715',
  'url': 'https://i.redd.it/20olbi48rpc51.jpg',
  'comms_num': 26,
  'timestamp': '2020-07-24T02:08:56+00:00',
  'body': '',
  'subscribers': 41729,
  'subreddit': 'umd'},
 {'author': 'None',
  'title': 'My INST633 prof told us to create OC posts for an assignment and we need 10K upvotes to get full credit please help a girl out, ty!',
  'score': 1027,
  'id': 'ku7wm4',
  'url': 'https://i.redd.it/y34tjot3tfa61.jpg',
  'comms_num': 25,
  'timestamp': '2021-01-10T05:02:05+00:00',
  'body': '',
  'subscribers': 41729,
  'subreddit': 'umd'},
 {'author': 

In [ ]:
# Now we want to export our JSON data back to our desktops
from google.colab import files

# Export reddit data to JSON and name the files appropriately
with open(subreddit_name + "_subreddit.json", 'w+') as f:
    json.dump(reddit_dict, f)

files.download(subreddit_name + "_subreddit.json") 
print("All data has been successfully exported to a JSON file.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All data has been successfully exported to a JSON file.
